In [1]:
import os
import csv
import MySQLdb
import EmailCreateSend
import DatabaseConnection as dbc
import LogViolation as l
pathName = os.getcwd()
index = []   #track image IDs 
logFiles = []
fileNames = os.listdir(pathName)

##Load all csv files in directory
##Script can scan multiple csv logs 
for fileNames in fileNames:
    if fileNames.endswith(".csv"):
        logFiles.append(fileNames)
        
##Loop for each log file
for i in logFiles:
    file = open(os.path.join(pathName, i), newline='')
    reader = csv.reader(file, delimiter=',')
    print('\n'+i)   ##print log file name 
    
    for row in reader:
        content = list(row[i] for i in [0,1]) ##[0,1] are included columns from CNN csv logs

        print (content)
        if float(content[1]) > .5:
            print('Violation')  
            index.append(content[0])


log-03_06_2019.csv
['1', '0.9']
Violation
['2', '0.64']
Violation
['3', '0']
['4', '0.051306013']
['5', '0']
['6', '0.022467662']


In [2]:
db = dbc.connect()
cursor = db.cursor()

In [3]:
query = "SELECT U.USER_ID, U.FNAME, U.LNAME, SS.SCREENSHOT_PATH, SS.URL FROM USERS U LEFT JOIN SCREENSHOTS SS ON U.USER_ID = SS.SS_USER WHERE SS.SS_ID IN (%s)" % ",".join(map(str,index))
cursor.execute(query)
results = cursor.fetchall()

In [4]:
message = ''
#EmailCreateSend.send(results)
for row in results:
    eid = row[0]
    fname = row[1]
    lname = row[2]
    ss = row[3]
    url = row[4]
    message += 'EMPLOYEE ID: ' + eid + ', NAME: ' + fname + ' ' + lname + ', URL: ' + url + '\n'
    ##LOG TO VIOLATION TABLE
    l.log(eid, url, ss)

In [5]:
print(message)

EMPLOYEE ID: 1, NAME: Test User, URL: www.cnn.com
EMPLOYEE ID: 2, NAME: Tyler Dugan, URL: www.cnn.com



In [6]:
EmailCreateSend.send(message)